In [ ]:
import os
import time
import glob
import cv2
import pytesseract
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Directory where images are captured
image_dir = "captured_images"               

# Tesseract configuration for better text extraction
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
tess_config = "--oem 3 --psm 6"  # Adjust as needed for your use case

def extract_text_from_image(image_path):
    """Reads an image and extracts text using Tesseract OCR."""
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    text = pytesseract.image_to_string(gray, config=tess_config)
    return text

print("Monitoring the directory for new images...")

while True:
    # Get all image files in the directory
    image_files = glob.glob(os.path.join(image_dir, "image-*.*"))

    if image_files:
        for img_path in image_files:
            print(f"Processing image: {img_path}")
            
            # Extract text from the image
            extracted_text = extract_text_from_image(img_path)

            # Print the extracted text
            if extracted_text.strip():
                print("Extracted Text:\n", extracted_text)
                
                # Display the image with Matplotlib
                img = cv2.imread(img_path)
                plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                plt.title('Extracted Text Preview')
                plt.axis('off')
                plt.show()
            else:
                print("No text detected.")

            # Clear the output to keep terminal clean
            clear_output(wait=True)
            time.sleep(3)  # Wait 3 seconds before next iteration

            # Optionally delete processed images to avoid reprocessing
            # os.remove(img_path)
            # print(f"Deleted image: {img_path}")
    
    else:
        print("No new images found. Checking again...")
        time.sleep(1)  # Wait 1 second before checking again


In [ ]:
import os
import time
import glob
import cv2
import pytesseract
import matplotlib.pyplot as plt
from IPython.display import clear_output
import serial  
import re  
from datetime import datetime

image_dir = "captured_images"

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
tess_config = "--oem 3 --psm 6"  

def extract_text_from_image(image_path):
    """Reads an image and extracts text using Tesseract OCR."""
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    text = pytesseract.image_to_string(gray, config=tess_config)
    return text

def extract_expiry_date(text):
    """Extracts a date from the text and checks if it's expired."""
    date_pattern = r'(\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b)'
    matches = re.findall(date_pattern, text)

    if matches:
        for match in matches:
            try:
                expiry_date = datetime.strptime(match, '%d/%m/%Y')
            except ValueError:
                try:
                    expiry_date = datetime.strptime(match, '%m/%d/%Y')
                except ValueError:
                    continue  

            if expiry_date < datetime.now():
                return True  # Expired

    return False  # Not expired or no valid date found

# Arduino setup
arduino = serial.Serial('COM11', 9600) 
time.sleep(2)  

print("Monitoring the directory for new images...")

while True:
    image_files = glob.glob(os.path.join(image_dir, "image-*.jpg"))

    if image_files:
        for img_path in image_files:
            print(f"Processing image: {img_path}")
            
            extracted_text = extract_text_from_image(img_path)

            if extracted_text.strip():
                print("Extracted Text:\n", extracted_text)

                if extract_expiry_date(extracted_text):
                    arduino.write(b'R')  # Run the motor for 2 seconds
                    time.sleep(2)  # Allow the motor to run
                    arduino.write(b'A')  # Activate the servo for sorting
                    print("Detected expired item. Motor run and servo activated.")
                else:
                    arduino.write(b'R')  # Run the motor for fresh items
                    print("Detected fresh item. Motor is running.")


                img = cv2.imread(img_path)
                plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                plt.title('Extracted Text Preview')
                plt.axis('off')
                plt.show()
            else:
                print("No text detected.")

            clear_output(wait=True)
            time.sleep(3)  # Wait 3 seconds before next iteration

            # os.remove(img_path)
            # print(f"Deleted image: {img_path}")
    
    else:
        print("No new images found. Checking again...")
        time.sleep(1)  # Wait 1 second before checking again
